In [9]:
#Import the required lib
import os
import numpy as np
import pandas as pd 
from utils import pct_change, preprocess_and_split_df, convert_to_dataset, data_preparation, plot_data, create_bollinger_limits
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import random
import torch
import statistics
from LSTM_model_future_change import LSTM_FutureChangeGeneral, LSTM_FutureChange

import warnings
warnings.filterwarnings('ignore')

## Upload the dataframes ## 

In [10]:
# create the data dict again 
CRYPTO_DICT = { 
            "ADA": {}, 
            "BCH": {},
            "BNB": {}, 
            "BTC": {},
            "ETH": {},
            "LTC": {}, 
            "NEO": {},
            "TRX": {},
            "XRP": {},
}

# upload the data 

for Crypto_code in CRYPTO_DICT:
    CRYPTO_DICT[Crypto_code]["df"]  = pd.read_pickle(f"Training_Data/Data_preprocessed/{Crypto_code}/{Crypto_code}-main_df.pkl")
    CRYPTO_DICT[Crypto_code]["df_train"]  = pd.read_pickle(f"Training_Data/Data_preprocessed/{Crypto_code}/{Crypto_code}-train_df.pkl")
    CRYPTO_DICT[Crypto_code]["df_val"]  = pd.read_pickle(f"Training_Data/Data_preprocessed/{Crypto_code}/{Crypto_code}-val_df.pkl")
    CRYPTO_DICT[Crypto_code]["df_test"]  = pd.read_pickle(f"Training_Data/Data_preprocessed/{Crypto_code}/{Crypto_code}-test_df.pkl")

In [11]:
CRYPTO_DICT["BTC"]["df_train"].columns

Index(['unix', 'date', 'symbol', 'close', 'mid_price_1', 'mid_price_2',
       'Volume USDT', 'SMA', 'EMA0', 'EMA1', 'EMA2', 'RSI', 'upper', 'lower',
       'upper_check', 'lower_check', 'future_price', 'future_change'],
      dtype='object', name=0)

## Create Datasets for all coins ##

In [12]:
# Dataset for future price prediction 
future_price_columns =  ['close', 'mid_price_1', 'mid_price_2',
       'Volume USDT', 'SMA', 'EMA0', 'EMA1', 'EMA2', 'RSI', 'upper', 'lower',
       'upper_check', 'lower_check', 'future_price']

future_price_length = (len(future_price_columns) - 1)

# Dataset for future change prediction 
future_change_columns = ['close', 'mid_price_1', 'mid_price_2',
       'Volume USDT', 'SMA', 'EMA0', 'EMA1', 'EMA2', 'RSI', 'upper', 'lower',
       'upper_check', 'lower_check','future_change']
future_change_length = (len(future_change_columns) -  1)

In [13]:
datasets = [["future_price_dataset", future_price_columns, future_price_length], ["future_change_dataset", future_change_columns, future_change_length]]
for dataset in datasets: 
    print(f"{datasets[0][0]} start converting for all coins")
    for Crypto_code in CRYPTO_DICT:
        if "datasets" not in list(CRYPTO_DICT[Crypto_code].keys()):
            CRYPTO_DICT[Crypto_code]["datasets"] = {}
        for df in CRYPTO_DICT[Crypto_code]: 
            if df == "df" or df == "datasets": 
                pass
            else:
                if dataset[0] not in list(CRYPTO_DICT[Crypto_code]["datasets"].keys()):
                    CRYPTO_DICT[Crypto_code]["datasets"][dataset[0]] = {}
                CRYPTO_DICT[Crypto_code]["datasets"][dataset[0]][f"{df}_dataset"] = convert_to_dataset(CRYPTO_DICT[Crypto_code][df], dataset[1], dataset[2])
    print(f"{datasets[0][0]} finished converting for all coins")

future_price_dataset start converting for all coins
future_price_dataset finished converting for all coins
future_price_dataset start converting for all coins
future_price_dataset finished converting for all coins


In [14]:
len(CRYPTO_DICT["ADA"]["datasets"]['future_change_dataset']['df_test_dataset'])

26733

In [15]:
len(CRYPTO_DICT["BTC"]["datasets"]['future_change_dataset']['df_test_dataset'])

24823

In [16]:
for key in CRYPTO_DICT: 
    print(key)

ADA
BCH
BNB
BTC
ETH
LTC
NEO
TRX
XRP


# Modeling section #

## 1. LSTM_MODEL ##

### 1.1 Future_change_prediction 

In [17]:
# Import model creation lib
import torch
from torch import nn
import pandas as pd
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter  # to print to tensorboard

#### Dataloaders and Hyperparameters ####

In [18]:
# Example of creating datasets for one coin 
# ADA 
# Process will be same for other coins 
train_dataset = CRYPTO_DICT["ADA"]["datasets"]['future_change_dataset']['df_train_dataset']
val_dataset = CRYPTO_DICT["ADA"]["datasets"]['future_change_dataset']['df_val_dataset']
test_dataset = CRYPTO_DICT["ADA"]["datasets"]['future_change_dataset']['df_test_dataset']

In [19]:
# training hyperparameters 
num_epoch = 6
batch_size = 64

# model hyperparameters 
input_size = len(train_dataset[0][0][0])
output_size = len(train_dataset[0][1])
hidden_size = 128
drop_out = 0.4
num_layers = 2

In [20]:
# Create dataloader for all data types
training_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
validation_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
testing_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
len(training_dataloader)

3348

#### Creating the model and checking for GPU ####

In [21]:
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

No GPU available, training on CPU; consider making n_epochs very small.


In [22]:
# Tensorboard to get nice loss plot
writer_train = SummaryWriter(f"runs/loss_plot")
writer_val = SummaryWriter(f"runs/val_plot")
writer_acc = SummaryWriter(f"runs/acc_plot")
step = 0

In [23]:
# Building the network
model = LSTM(input_size, hidden_size, num_layers, drop_out, output_size)
nn.MSELoss()
criterion = nn.BCELoss()

model = model.float()
model

LSTM(
  (dropout): Dropout(p=0.4, inplace=False)
  (lstm): LSTM(13, 128, num_layers=2, batch_first=True, dropout=0.4)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (batch1d2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch1d3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

#### Training loop and accuracy calculation ####

In [24]:
def acc_get(targets, output):
    output_round = torch.round(output)
    acc = 0
    for i in range(len(targets)): 
        if targets[i] == output_round[i]: 
            acc += 1 
    acc_pct = (acc) / len(targets)
    return acc_pct

In [25]:
def train(model, Crypto_code, training_dataloader, validation_dataloader, epochs=10, clip=2, print_every=200, learning_rate_decay=1400):
    step = 0 
    model.train()
    if(train_on_gpu):
        net.cuda()
    counter = 0 
    # start at inf for val_loss
    valid_loss_min = np.Inf
    
    # lr set at the beginning to be high at 0.1
    lr = 0.1
    opt = optim.SGD(model.parameters(), lr=lr)
    print(f"learning_rate: {lr}")
    
    # start training loop
    for e in range(epochs):
        # initialize hidden state
        h = model.init_hidden(batch_size, train_on_gpu)
        
        for inputs, targets in training_dataloader:
            counter += 1
            
            # decrease the learning rate every 1500 step 
            if counter % learning_rate_decay == 0: 
                # decrease the lr by 0.25 each 1500 step to help model converge 
                lr= lr * 0.725
                opt = optim.SGD(model.parameters(), lr=lr)
                print(f"learning_rate decreased to: {lr}")
                
            
            # move to GPU if available   
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()
                
            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])
            
            # zero accumulated gradients
            model.zero_grad()
            
            # get network output
            output, hidden = model(inputs, h)
            # find loss and back propogate 
            loss = criterion(output, targets.float())
            loss.backward()
            
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(model.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_acc = []
                val_h = model.init_hidden(batch_size, train_on_gpu)
                val_losses = []
                model.eval()
                for inputs, targets in validation_dataloader: 
                    
                    # move to GPU
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()
                    
                    # create new hidden varaibles 
                    val_h = tuple([each.data for each in val_h])
                    
                    # forward 
                    output, val_h = model(inputs, val_h)
                    
                    # calculate val_batch accuracy 
                    val_acc.append(acc_get(targets, output))
                    
                    # loss
                    val_loss = criterion(output, targets.float())
                    val_losses.append(val_loss.item())
                
                model.train()
                
                # average val loss over all batches 
                avg_val_loss = np.mean(val_losses)
                
                # avg val accuracy over all batches 
                avg_val_acc = statistics.mean(val_acc)
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)),
                  "val_acc: {:.6f}".format(avg_val_acc))
                    
                    
                writer_train.add_scalar('training loss', loss.item(), global_step=step)
                writer_val.add_scalar('val_loss', avg_val_loss, global_step=step)
                writer_acc.add_scalar('val_accuracy', avg_val_acc, global_step=step)
                step += 1
                
                # save model if validation loss has decreased
                if avg_val_loss <= valid_loss_min:
                    print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                    valid_loss_min,
                    avg_val_loss))
                    torch.save(model.state_dict(), f"Trained-models/LSTM-model/{Crypto_code}-LSTM_model.pt")
                    valid_loss_min = avg_val_loss

### Model Testing on Testing Datasets ###

In [13]:
# Upload the general model to be compared by each coin-specific model 
MAIN_MODEL = LSTM_FutureChangeGeneral(input_size, 160, num_layers, 0.45, output_size).float()
MAIN_MODEL.load_state_dict(torch.load("Trained-models/LSTM-model/General-LSTM_model.pt", map_location='cpu'))

<All keys matched successfully>

In [19]:
# Loop for test data evaulation for each coin 
models = [MAIN_MODEL, None]
criterion = nn.BCELoss()
for Crypto_code in CRYPTO_DICT:
    print(f"###### Start testing for {Crypto_code} ########")
    
    # Create the Dataloaders 
    test_dataset = CRYPTO_DICT[Crypto_code]["datasets"]['future_change_dataset']['df_train_dataset']
    testing_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    print(f"Number of batches for {Crypto_code}-Coin: {len(testing_dataloader)}")
    
    
    
    # upload the coin specific model 
    coin_model = LSTM_FutureChange(input_size, hidden_size, num_layers, drop_out, output_size).float()
    coin_model.load_state_dict(torch.load(f"Trained-models/LSTM-model/{Crypto_code}-LSTM_model.pt"))
    models[1] = coin_model
    
    # start testing for each model
    for i_model, (model) in enumerate(models): 
        test_acc = []
        test_h = model.init_hidden(batch_size, train_on_gpu)
        test_losses = []
        model.eval()
        for i, (inputs, targets) in enumerate(testing_dataloader): 
        #     print(i)
            # move to GPU
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # create new hidden varaibles 
            test_h = tuple([each.data for each in test_h])

            # forward 
            output, test_h = model(inputs, test_h)

            # calculate val_batch accuracy 
            test_acc.append(acc_get(targets, output))

            # loss
            test_loss = criterion(output, targets.float())
            test_losses.append(test_loss.item())
        
        if i_model == 0: 
        
            # average val loss over all batches 
            avg_test_loss = np.mean(test_losses)

            # avg val accuracy over all batches 
            avg_test_acc = statistics.mean(test_acc)
            print(f"test losses for {Crypto_code} using General_Model: {avg_test_loss}")
            print(f"test accuracy for {Crypto_code} using General_Model: {avg_test_acc * 100}%")
        else: 
            # average val loss over all batches 
            avg_test_loss = np.mean(test_losses)

            # avg val accuracy over all batches 
            avg_test_acc = statistics.mean(test_acc)
            print(f"test losses for {Crypto_code} using coin_specific_model: {avg_test_loss}")
            print(f"test accuracy for {Crypto_code} using coin_specific_model: {avg_test_acc * 100}%")


###### Start testing for ADA ########
Number of batches for ADA-Coin: 3348
test losses for ADA using General_Model: 0.2993612183099935
test accuracy for ADA using General_Model: 89.96602449223417%
test losses for ADA using coin_specific_model: 0.32181599406412
test accuracy for ADA using coin_specific_model: 88.49919728195937%
###### Start testing for BCH ########
Number of batches for BCH-Coin: 2598
test losses for BCH using General_Model: 0.4905321517180626
test accuracy for BCH using General_Model: 81.20729888375674%
test losses for BCH using coin_specific_model: 0.45796974766153475
test accuracy for BCH using coin_specific_model: 78.99345650500385%
###### Start testing for BNB ########
Number of batches for BNB-Coin: 2572
test losses for BNB using General_Model: 0.2788405732293707
test accuracy for BNB using General_Model: 90.76533339813375%
test losses for BNB using coin_specific_model: 0.28445740154774896
test accuracy for BNB using coin_specific_model: 89.5922433903577%
###### S

In [92]:
len(validation_dataloader)

417

## 2. Model using future price

In [7]:
# Example of creating datasets for one coin 
# ADA 
# Process will be same for other coins 
train_dataset = CRYPTO_DICT["ADA"]["datasets"]['future_price_dataset']['df_train_dataset']
val_dataset = CRYPTO_DICT["ADA"]["datasets"]['future_price_dataset']['df_val_dataset']
test_dataset = CRYPTO_DICT["ADA"]["datasets"]['future_price_dataset']['df_test_dataset']
len(test_dataset)

NameError: name 'CRYPTO_DICT' is not defined

In [8]:
# training hyperparameters 
num_epoch = 8
batch_size = 64

# model hyperparameters 
input_size = len(train_dataset[0][0][0])
output_size = len(train_dataset[0][1])
hidden_size = 128
drop_out = 0.4
num_layers = 2

NameError: name 'train_dataset' is not defined

In [24]:
# Create dataloader for all data types
training_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
validation_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
testing_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
len(training_dataloader)

3348

In [26]:
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

No GPU available, training on CPU; consider making n_epochs very small.


In [6]:
import torch.nn as nn
import torch.nn.functional as F


class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout_ratio, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.input_size = input_size
        self.output_size = output_size

        # dropout layer
        self.dropout = nn.Dropout(dropout_ratio)

        # LSTM layer
        self.lstm = nn.LSTM(self.input_size, self.hidden_size, num_layers=self.num_layers, dropout=dropout_ratio,
                            batch_first=True)

        # fully connected layer
        self.fc1 = nn.Linear(hidden_size, 64)
        self.fc2 = nn.Linear(64, self.output_size)

        # # BATCH_NORMALIZATION layer
 
        self.batch1d2 = nn.BatchNorm1d(hidden_size)
        self.batch1d3 = nn.BatchNorm1d(64)

    def forward(self, x, hidden):

        batch_size = x.size(0)

        # lstm layer x[B, seq_len, input_size]
        x, hidden = self.lstm(x, hidden)

        # get the last output
        x = x[:, -1, :]

        # drop out and batch_norm
        x = self.dropout(x)
        x = self.batch1d2(x)

        # first fc layer
        x = F.leaky_relu(self.fc1(x))

        # drop out and batch_norm
        x = self.dropout(x)
        x = self.batch1d3(x)

        # first fc layer
        x = self.fc2(x)

        return x, hidden

    def init_hidden(self, batch_size, train_on_gpu):
        '''
        Initializes hidden state
        '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        if (train_on_gpu):
            hidden = (weight.new(self.num_layers, batch_size, self.hidden_size).zero_().cuda(),
                      weight.new(self.num_layers, batch_size, self.hidden_size).zero_().cuda())
        else:
            hidden = (weight.new(self.num_layers, batch_size, self.hidden_size).zero_(),
                      weight.new(self.num_layers, batch_size, self.hidden_size).zero_())

        return hidden


In [34]:
# Building the network
model = LSTM(input_size, hidden_size, num_layers, drop_out, output_size)

criterion = nn.MSELoss()

model = model.float()
model

LSTM(
  (dropout): Dropout(p=0.4, inplace=False)
  (lstm): LSTM(13, 128, num_layers=2, batch_first=True, dropout=0.4)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (batch1d2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch1d3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [27]:
def train(model, Crypto_code, training_dataloader, validation_dataloader, epochs=10, clip=2, print_every=200, learning_rate_decay=1800):
    step = 0 
    model.train()
    if(train_on_gpu):
        net.cuda()
    counter = 0 
    # start at inf for val_loss
    valid_loss_min = np.Inf
    
    # lr set at the beginning to be high at 0.1
    lr = 0.01
    opt = optim.SGD(model.parameters(), lr=lr)
    print(f"learning_rate: {lr}")
    
    # start training loop
    for e in range(epochs):
        # initialize hidden state
        h = model.init_hidden(batch_size, train_on_gpu)
        
        for inputs, targets in training_dataloader:
            counter += 1
            
            # decrease the learning rate every 1500 step 
            if counter % learning_rate_decay == 0: 
                # decrease the lr by 0.25 each 1500 step to help model converge 
                lr= lr * 0.775
                opt = optim.SGD(model.parameters(), lr=lr)
                print(f"learning_rate decreased to: {lr}")
                
            
            # move to GPU if available   
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()
                
            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])
            
            # zero accumulated gradients
            model.zero_grad()
            
            # get network output
            output, hidden = model(inputs, h)
            # find loss and back propogate 
            loss = criterion(output, targets.float())
            loss.backward()
            
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(model.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = model.init_hidden(batch_size, train_on_gpu)
                val_losses = []
                model.eval()
                for inputs, targets in validation_dataloader: 
                    
                    # move to GPU
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()
                    
                    # create new hidden varaibles 
                    val_h = tuple([each.data for each in val_h])
                    
                    # forward 
                    output, val_h = model(inputs, val_h)
                    
                    # calculate val_batch accuracy 
                    
                    # loss
                    val_loss = criterion(output, targets.float())
                    val_losses.append(val_loss.item())
                
                model.train()
                
                # average val loss over all batches 
                avg_val_loss = np.mean(val_losses)
                
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))
                    
                
                # save model if validation loss has decreased
                if avg_val_loss <= valid_loss_min:
                    print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                    valid_loss_min,
                    avg_val_loss))
                    torch.save(model.state_dict(), f"Trained-models/LSTM-model/Future_price/{Crypto_code}_price_LSTM_model.pt")
                    valid_loss_min = avg_val_loss

In [43]:
Crypto_code = "ADA"
train(model, Crypto_code, training_dataloader, validation_dataloader, epochs= num_epoch, print_every=200)

learning_rate: 0.01
Epoch: 1/8... Step: 200... Loss: 1.352781... Val Loss: 0.649328
Validation loss decreased (inf --> 0.649328).  Saving model ...
Epoch: 1/8... Step: 400... Loss: 0.932932... Val Loss: 0.649976
Epoch: 1/8... Step: 600... Loss: 0.635322... Val Loss: 0.650135
Epoch: 1/8... Step: 800... Loss: 0.763163... Val Loss: 0.649900
Epoch: 1/8... Step: 1000... Loss: 1.297694... Val Loss: 0.648800
Validation loss decreased (0.649328 --> 0.648800).  Saving model ...
Epoch: 1/8... Step: 1200... Loss: 0.958415... Val Loss: 0.654124
Epoch: 1/8... Step: 1400... Loss: 0.871864... Val Loss: 0.649502
Epoch: 1/8... Step: 1600... Loss: 1.301817... Val Loss: 0.649969
learning_rate decreased to: 0.007750000000000001
Epoch: 1/8... Step: 1800... Loss: 0.911590... Val Loss: 0.650423
Epoch: 1/8... Step: 2000... Loss: 0.728262... Val Loss: 0.649902
Epoch: 1/8... Step: 2200... Loss: 0.876958... Val Loss: 0.649242
Epoch: 1/8... Step: 2400... Loss: 0.920017... Val Loss: 0.649658
Epoch: 1/8... Step: 26

Epoch: 6/8... Step: 17800... Loss: 0.792146... Val Loss: 0.489002
Validation loss decreased (0.492686 --> 0.489002).  Saving model ...
learning_rate decreased to: 0.0007816584462936415
Epoch: 6/8... Step: 18000... Loss: 0.638651... Val Loss: 0.489036
Epoch: 6/8... Step: 18200... Loss: 0.882995... Val Loss: 0.484895
Validation loss decreased (0.489002 --> 0.484895).  Saving model ...
Epoch: 6/8... Step: 18400... Loss: 0.984945... Val Loss: 0.484580
Validation loss decreased (0.484895 --> 0.484580).  Saving model ...
Epoch: 6/8... Step: 18600... Loss: 0.724240... Val Loss: 0.485858
Epoch: 6/8... Step: 18800... Loss: 0.717596... Val Loss: 0.484665
Epoch: 6/8... Step: 19000... Loss: 0.971033... Val Loss: 0.482775
Validation loss decreased (0.484580 --> 0.482775).  Saving model ...
Epoch: 6/8... Step: 19200... Loss: 0.740823... Val Loss: 0.485715
Epoch: 6/8... Step: 19400... Loss: 0.912307... Val Loss: 0.488895
Epoch: 6/8... Step: 19600... Loss: 0.718699... Val Loss: 0.484372
learning_rate 

In [44]:
Crypto_code = "BCH"
train_dataset = CRYPTO_DICT[Crypto_code]["datasets"]['future_price_dataset']['df_train_dataset']
val_dataset = CRYPTO_DICT[Crypto_code]["datasets"]['future_price_dataset']['df_val_dataset']
test_dataset = CRYPTO_DICT[Crypto_code]["datasets"]['future_price_dataset']['df_test_dataset']

training_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
validation_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
testing_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
print(training_dataloader)


# Building the network
model = LSTM(input_size, hidden_size, num_layers, drop_out, output_size)

criterion = nn.MSELoss()

model = model.float()
model

LSTM(
  (dropout): Dropout(p=0.4, inplace=False)
  (lstm): LSTM(13, 128, num_layers=2, batch_first=True, dropout=0.4)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (batch1d2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch1d3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [45]:
train(model, Crypto_code, training_dataloader, validation_dataloader, epochs= num_epoch, print_every=200)

learning_rate: 0.01
Epoch: 1/8... Step: 200... Loss: 0.716425... Val Loss: 0.614430
Validation loss decreased (inf --> 0.614430).  Saving model ...
Epoch: 1/8... Step: 400... Loss: 1.112655... Val Loss: 0.607076
Validation loss decreased (0.614430 --> 0.607076).  Saving model ...
Epoch: 1/8... Step: 600... Loss: 1.189215... Val Loss: 0.598043
Validation loss decreased (0.607076 --> 0.598043).  Saving model ...
Epoch: 1/8... Step: 800... Loss: 1.553865... Val Loss: 0.610326
Epoch: 1/8... Step: 1000... Loss: 1.204416... Val Loss: 0.560056
Validation loss decreased (0.598043 --> 0.560056).  Saving model ...
Epoch: 1/8... Step: 1200... Loss: 0.849272... Val Loss: 0.815143
Epoch: 1/8... Step: 1400... Loss: 1.148699... Val Loss: 0.522267
Validation loss decreased (0.560056 --> 0.522267).  Saving model ...
Epoch: 1/8... Step: 1600... Loss: 0.989617... Val Loss: 0.528071
learning_rate decreased to: 0.007750000000000001
Epoch: 1/8... Step: 1800... Loss: 1.063417... Val Loss: 0.501296
Validation

Epoch: 8/8... Step: 19800... Loss: 0.994365... Val Loss: 0.450611
Epoch: 8/8... Step: 20000... Loss: 0.600391... Val Loss: 0.445156
Validation loss decreased (0.446432 --> 0.445156).  Saving model ...
Epoch: 8/8... Step: 20200... Loss: 0.544400... Val Loss: 0.448544
Epoch: 8/8... Step: 20400... Loss: 0.748593... Val Loss: 0.448201
Epoch: 8/8... Step: 20600... Loss: 0.989294... Val Loss: 0.446980


In [46]:
Crypto_code = "BNB"
train_dataset = CRYPTO_DICT[Crypto_code]["datasets"]['future_price_dataset']['df_train_dataset']
val_dataset = CRYPTO_DICT[Crypto_code]["datasets"]['future_price_dataset']['df_val_dataset']
test_dataset = CRYPTO_DICT[Crypto_code]["datasets"]['future_price_dataset']['df_test_dataset']

training_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
validation_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
testing_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
print(training_dataloader)


# Building the network
model = LSTM(input_size, hidden_size, num_layers, drop_out, output_size)

criterion = nn.MSELoss()

model = model.float()
model

LSTM(
  (dropout): Dropout(p=0.4, inplace=False)
  (lstm): LSTM(13, 128, num_layers=2, batch_first=True, dropout=0.4)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (batch1d2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch1d3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [ ]:
train(model, Crypto_code, training_dataloader, validation_dataloader, epochs= num_epoch, print_every=200)

learning_rate: 0.01
Epoch: 1/8... Step: 200... Loss: 0.669767... Val Loss: 0.655472
Validation loss decreased (inf --> 0.655472).  Saving model ...
Epoch: 1/8... Step: 400... Loss: 0.776793... Val Loss: 0.620723
Validation loss decreased (0.655472 --> 0.620723).  Saving model ...
Epoch: 1/8... Step: 600... Loss: 1.102229... Val Loss: 0.579936
Validation loss decreased (0.620723 --> 0.579936).  Saving model ...
Epoch: 1/8... Step: 800... Loss: 1.318161... Val Loss: 0.664424
Epoch: 1/8... Step: 1000... Loss: 0.847560... Val Loss: 0.753537
Epoch: 1/8... Step: 1200... Loss: 1.137098... Val Loss: 0.518982
Validation loss decreased (0.579936 --> 0.518982).  Saving model ...
Epoch: 1/8... Step: 1400... Loss: 0.976274... Val Loss: 0.501454
Validation loss decreased (0.518982 --> 0.501454).  Saving model ...
Epoch: 1/8... Step: 1600... Loss: 0.879374... Val Loss: 0.493177
Validation loss decreased (0.501454 --> 0.493177).  Saving model ...
learning_rate decreased to: 0.007750000000000001
Epoch:

Epoch: 7/8... Step: 16600... Loss: 0.955581... Val Loss: 0.387345
Epoch: 7/8... Step: 16800... Loss: 0.831022... Val Loss: 0.388212
Epoch: 7/8... Step: 17000... Loss: 0.781578... Val Loss: 0.377489
Epoch: 7/8... Step: 17200... Loss: 0.578804... Val Loss: 0.376047
Epoch: 7/8... Step: 17400... Loss: 0.715403... Val Loss: 0.374685
Validation loss decreased (0.375461 --> 0.374685).  Saving model ...
Epoch: 7/8... Step: 17600... Loss: 0.798240... Val Loss: 0.374467
Validation loss decreased (0.374685 --> 0.374467).  Saving model ...
Epoch: 7/8... Step: 17800... Loss: 0.698832... Val Loss: 0.376729
learning_rate decreased to: 0.0007816584462936415
Epoch: 7/8... Step: 18000... Loss: 0.798257... Val Loss: 0.373075
Validation loss decreased (0.374467 --> 0.373075).  Saving model ...
Epoch: 8/8... Step: 18200... Loss: 0.715390... Val Loss: 0.369105
Validation loss decreased (0.373075 --> 0.369105).  Saving model ...
Epoch: 8/8... Step: 18400... Loss: 0.620352... Val Loss: 0.380769
Epoch: 8/8... 

In [ ]:
Crypto_code = "BTC"
train_dataset = CRYPTO_DICT[Crypto_code]["datasets"]['future_price_dataset']['df_train_dataset']
val_dataset = CRYPTO_DICT[Crypto_code]["datasets"]['future_price_dataset']['df_val_dataset']
test_dataset = CRYPTO_DICT[Crypto_code]["datasets"]['future_price_dataset']['df_test_dataset']

training_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
validation_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
testing_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
print(training_dataloader)


# Building the network
model = LSTM(input_size, hidden_size, num_layers, drop_out, output_size)

criterion = nn.MSELoss()

model = model.float()
model

In [ ]:
train(model, Crypto_code, training_dataloader, validation_dataloader, epochs= num_epoch, print_every=200)

In [ ]:
Crypto_code = "ETH"
train_dataset = CRYPTO_DICT[Crypto_code]["datasets"]['future_price_dataset']['df_train_dataset']
val_dataset = CRYPTO_DICT[Crypto_code]["datasets"]['future_price_dataset']['df_val_dataset']
test_dataset = CRYPTO_DICT[Crypto_code]["datasets"]['future_price_dataset']['df_test_dataset']

training_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
validation_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
testing_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
print(training_dataloader)


# Building the network
model = LSTM(input_size, hidden_size, num_layers, drop_out, output_size)

criterion = nn.MSELoss()

model = model.float()
model

In [ ]:
train(model, Crypto_code, training_dataloader, validation_dataloader, epochs= num_epoch, print_every=200)

In [ ]:
Crypto_code = "LTC"
train_dataset = CRYPTO_DICT[Crypto_code]["datasets"]['future_price_dataset']['df_train_dataset']
val_dataset = CRYPTO_DICT[Crypto_code]["datasets"]['future_price_dataset']['df_val_dataset']
test_dataset = CRYPTO_DICT[Crypto_code]["datasets"]['future_price_dataset']['df_test_dataset']

training_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
validation_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
testing_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
print(training_dataloader)


# Building the network
model = LSTM(input_size, hidden_size, num_layers, drop_out, output_size)

criterion = nn.MSELoss()

model = model.float()
model

In [ ]:
train(model, Crypto_code, training_dataloader, validation_dataloader, epochs= num_epoch, print_every=200)

In [ ]:
Crypto_code = "NEO"
train_dataset = CRYPTO_DICT[Crypto_code]["datasets"]['future_price_dataset']['df_train_dataset']
val_dataset = CRYPTO_DICT[Crypto_code]["datasets"]['future_price_dataset']['df_val_dataset']
test_dataset = CRYPTO_DICT[Crypto_code]["datasets"]['future_price_dataset']['df_test_dataset']

training_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
validation_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
testing_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
print(training_dataloader)


# Building the network
model = LSTM(input_size, hidden_size, num_layers, drop_out, output_size)

criterion = nn.MSELoss()

model = model.float()
model

In [ ]:
train(model, Crypto_code, training_dataloader, validation_dataloader, epochs= num_epoch, print_every=200)

In [ ]:
Crypto_code = "TRX"
train_dataset = CRYPTO_DICT[Crypto_code]["datasets"]['future_price_dataset']['df_train_dataset']
val_dataset = CRYPTO_DICT[Crypto_code]["datasets"]['future_price_dataset']['df_val_dataset']
test_dataset = CRYPTO_DICT[Crypto_code]["datasets"]['future_price_dataset']['df_test_dataset']

training_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
validation_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
testing_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
print(training_dataloader)


# Building the network
model = LSTM(input_size, hidden_size, num_layers, drop_out, output_size)

criterion = nn.MSELoss()

model = model.float()
model

In [ ]:
train(model, Crypto_code, training_dataloader, validation_dataloader, epochs= num_epoch, print_every=200)

In [ ]:
Crypto_code = "XRP"
train_dataset = CRYPTO_DICT[Crypto_code]["datasets"]['future_price_dataset']['df_train_dataset']
val_dataset = CRYPTO_DICT[Crypto_code]["datasets"]['future_price_dataset']['df_val_dataset']
test_dataset = CRYPTO_DICT[Crypto_code]["datasets"]['future_price_dataset']['df_test_dataset']

training_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
validation_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
testing_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
print(training_dataloader)


# Building the network
model = LSTM(input_size, hidden_size, num_layers, drop_out, output_size)

criterion = nn.MSELoss()

model = model.float()
model

In [ ]:
train(model, Crypto_code, training_dataloader, validation_dataloader, epochs= num_epoch, print_every=200)

In [29]:
model = LSTM(input_size, hidden_size, num_layers, drop_out, output_size)
model.load_state_dict(torch.load("Trained-models/LSTM-model/Future_price/BNB_price_LSTM_model.pt"))


<All keys matched successfully>

In [30]:
test_dataset = CRYPTO_DICT["BNB"]["datasets"]['future_price_dataset']['df_test_dataset']

In [31]:
testing_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

In [38]:
model.eval()
val_h = model.init_hidden(batch_size, train_on_gpu)
for inputs, targets in testing_dataloader: 

    # move to GPU
    if(train_on_gpu):
        inputs, targets = inputs.cuda(), targets.cuda()

    # create new hidden varaibles 
    val_h = tuple([each.data for each in val_h])

    # forward 
    output, val_h = model(inputs, val_h)

    # calculate val_batch accuracy 

    print([output[:5], targets[:5]])
    break
                

[tensor([[ 0.4999],
        [-1.0107],
        [-0.5108],
        [ 0.8360],
        [ 0.3761]], grad_fn=<SliceBackward>), tensor([[ 0.6384],
        [-2.7959],
        [-0.3228],
        [ 1.2012],
        [ 1.2950]])]
